## Projeto 2


### Como microdados que descrevem um magistrado, a exemplo de seu cargo, região e lotação, afetam sua renda total


---------

#### Lendo a base de dados


In [2]:
import pandas as pd
from  IPython.display import display
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt
import statistics as stats
import re

In [19]:
df = pd.read_csv("DF/contracheque.csv")
df = df.drop(columns=['cpf']) #Retirando a coluna CPF, pois os valores eram NaN.
df = df.drop(columns=['url']) #Retirando a coliuna URL, pois ela não é muito útil.
df.tail(1)

,nome,cargo,lotacao,subsidio,direitos_pessoais,indenizacoes,direitos_eventuais,total_de_rendimentos,previdencia_publica,imposto_de_renda,descontos_diversos,retencao_por_teto_constitucional,total_de_descontos,rendimento_liquido,remuneracao_do_orgao_de_origem,diarias,tribunal,orgao,data_de_publicacao,mesano_de_referencia
161766,Romeu Martinelli,Juiz Civil de 2º Grau,Aposentado,30471.11,0.0,0.0,85987.9,116459.01,5494.49,5974.28,0.0,0.0,11468.77,104990.24,0.0,0.0,Tribunal de Justiça Militar do Estado do Rio G...,Tribunal de Justiça Militar do Rio Grande do Sul,2017-12-06,2017-11-01


### Glossário das Colunas

#### Com o melhor entendimento do leitor em mente, segue a explicação resumida do que cada coluna mais intrincada do dataframe implica:



Variável | Descrição
:---:|---:
cargo | função que o invidíduo exerce no poder judiciário
lotação | jurisdição do magistrado dentro de seu cargo.
subsidio | remuneração mensal fixa dos magistrados e autoridades do setor público
direitos_pessoais | benefícios conseguidos pelo governo, auxílios
indenizações | valor pago ao magistrado para ressarcimento de despesas do serviço
direitos_eventuais | pagamentos incomuns que náo fazem parte da remuneração usual, como férias não aproveitadas
TOTAL DE RENDIMENTOS | montante total recebido pelo indivíduo
previdência pública | valor descontado do salário para cobrir aposentadoria futura
imposto de renda | desconto sobre rendimento mensal
descontos diversos | outros descontos no geral
retenção por teto constitucional | refere-se ao limite máximo de salário ditado pela constituição
RENDIMENTO LÍQUIDO | Montante recebido após feitos todos os descontos
remuneração do órgão de origem | valor bruto da remuneração do funcionário, antes de quaisquer descontos ou benefícios.
diárias | valor recebido para cobrir despesas diversas de viagens e trabalho
Tribunal e Órgão | Instituição à qual o indivíduo empresta seus serviços, seja esta um tribunal ou órgão



In [16]:
# Já de início, veremos se uma mesma pessoa pode aparecer mais de uma vez no dataframe


Celina = df.loc[df['nome'] == 'CELINA DIETRICH E TRIGUEIROS TEIXEIRA PINTO']

print (len(Celina))
Celina.describe()


11


,subsidio,direitos_pessoais,indenizacoes,direitos_eventuais,total_de_rendimentos,previdencia_publica,imposto_de_renda,descontos_diversos,retencao_por_teto_constitucional,total_de_descontos,rendimento_liquido,remuneracao_do_orgao_de_origem,diarias
count,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.0,11.000000,11.000000,11.0,11.0
mean,13157.977273,828.075455,2361.695455,14508.482727,30856.230909,1592.115455,3620.263636,1344.433636,0.0,6556.812727,24299.418182,0.0,0.0
std,15117.364949,2746.415581,2715.313454,14537.422356,30856.524948,1883.816718,3973.252549,4384.554160,0.0,9361.506821,23649.743432,0.0,0.0
min,0.000000,0.000000,0.000000,-154.390000,-154.390000,0.000000,0.000000,0.000000,0.0,0.000000,-154.390000,0.0,0.0
25%,0.000000,0.000000,0.000000,4477.220000,4554.415000,0.000000,388.610000,0.000000,0.0,388.610000,4183.170000,0.0,0.0
50%,0.000000,0.000000,0.000000,4631.610000,9108.830000,0.000000,1635.570000,0.000000,0.0,1635.570000,7473.260000,0.0,0.0
75%,28947.550000,0.000000,5232.730000,24639.300000,58884.580000,3184.230000,6111.270000,45.000000,0.0,9340.500000,46927.315000,0.0,0.0
max,28947.550000,9108.830000,5297.730000,42007.930000,75873.210000,4776.350000,12222.540000,14563.770000,0.0,31562.660000,57063.360000,0.0,0.0


#### Como vemos acima, um mesmo magistrado aparece diversas vezes no dataframe. Deve-se isso ao tamanho da amostra, mas esse fato não só não é um problema como nos beneficia.

#### Visto que o dataframe foi atualizado em meses diferentes, cada discrepância individual significa muito menos no todo. Além disso, os dados também revelam salários por posições diversas na magistratura para a mesma pessoa e área, o que nos ajudará a diferenciar a remuneração de cada cargo

-----------------------------------------

-----------

### **Agora, para que possamos posteriormente analisar os magistrados por local de atuação, dividiremos os tribunais por estado de origem, também elencando cada estado por região geográfica**

In [ ]:
# Idealizando uma função que nos devolva o estado e região nos quais um tribunal está inserido.

def Local_Tribunal(tribunal):
    for regiao, estados in regioes.items():
        for estado in estados:
            if tribunal in estados[e]:
                return (estado, regiao)
    return None

# Aplique essa função ao seu DataFrame para criar a nova coluna de região
df['regiao'] = df['estado'].apply(Local_Tribunal)  # Supondo 'estado' como a coluna com os estados


# Criaremos um grande dicionário único, que separa os Tribunais por estados nos quais estão localizados e esses estados pelas respectivas regiões geográficas.